## Extraction

In [122]:
%matplotlib inline

In [123]:
import numpy as np
import pandas as pd
from datetime import datetime

# import xlrd
# import matplotlib as mpl
import matplotlib.pyplot as plt
# import seaborn as sns
from pandas.api.types import CategoricalDtype

# For import export of model
import pickle

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# print(plt.style.available)
# mpl.style.use(['seaborn']) # optional: for ggplot-like style


In [124]:
# unidecode(sys.argv[1])

with open('longevity_template.pickle', 'rb') as f:
    fragrance, profile, weather = pickle.load(f)

In [125]:
fragrance

'{"fragrance":"Heint","fragrance_gender":"Female","fragrance_type":"Eau Fraiche","brand":"Venom","brand_tier":"High"}'

In [126]:
# fragrance_df = pd.read_json(fragrance)

fragrance_df    =  pd.DataFrame(data=eval(fragrance), index=[0])
profile_df      =  pd.DataFrame(data=eval(profile),   index=[0])
weather_df      =  pd.DataFrame(data=eval(weather),   index=[0])

In [127]:
fragrance_df
profile_df
weather_df

,temp,hum,dew_point,uv_index,temp_feels_like,atm_pressure,clouds,visibility,wind_speed,rain,snow,weather_main,weather_desc
0,82.975,75,74.45125,13.2475,89.745,1010.75,85.75,60000,11.01875,2.69375,0,Rain,light rain


In [128]:
df = pd.concat([fragrance_df, profile_df, weather_df], axis=1)

In [129]:
df

,fragrance,fragrance_gender,fragrance_type,brand,brand_tier,fp_id,gender,dob,sweat,height,...,uv_index,temp_feels_like,atm_pressure,clouds,visibility,wind_speed,rain,snow,weather_main,weather_desc
0,Heint,Female,Eau Fraiche,Venom,High,1,Male,2019-11-12,82,54,...,13.2475,89.745,1010.75,85.75,60000,11.01875,2.69375,0,Rain,light rain


In [130]:
df.head()

,fragrance,fragrance_gender,fragrance_type,brand,brand_tier,fp_id,gender,dob,sweat,height,...,uv_index,temp_feels_like,atm_pressure,clouds,visibility,wind_speed,rain,snow,weather_main,weather_desc
0,Heint,Female,Eau Fraiche,Venom,High,1,Male,2019-11-12,82,54,...,13.2475,89.745,1010.75,85.75,60000,11.01875,2.69375,0,Rain,light rain


## Cleaning

In [131]:
# df = df[df.columns.difference(['fba_country_name', 'fba_time_zone', 'suitability', 'sustainability', 'sillage', 'like', 'users_id', 'users_check', 'fba_location_country', 'fba_location_zone'], sort=False)]
df = df[df.columns.difference(['weather_desc'], sort=False)]

In [132]:
df

,fragrance,fragrance_gender,fragrance_type,brand,brand_tier,fp_id,gender,dob,sweat,height,...,dew_point,uv_index,temp_feels_like,atm_pressure,clouds,visibility,wind_speed,rain,snow,weather_main
0,Heint,Female,Eau Fraiche,Venom,High,1,Male,2019-11-12,82,54,...,74.45125,13.2475,89.745,1010.75,85.75,60000,11.01875,2.69375,0,Rain


In [133]:
df.dtypes

fragrance            object
fragrance_gender     object
fragrance_type       object
brand                object
brand_tier           object
fp_id                 int64
gender               object
dob                  object
sweat                 int64
height                int64
weight                int64
profession           object
skin_type            object
season               object
fp_country           object
temp                float64
hum                   int64
dew_point           float64
uv_index            float64
temp_feels_like     float64
atm_pressure        float64
clouds              float64
visibility            int64
wind_speed          float64
rain                float64
snow                  int64
weather_main         object
dtype: object

In [134]:
# df.isnull().sum()

In [135]:
# df['rain']

In [136]:
# Missing Values
# df['suitability'].fillna(df['suitability'].median(), inplace=True)
# df['suitability'].fillna(int(df['suitability'].mean()), inplace=True)

# df['suitability'].fillna(int(0), inplace=True)

# df['projection'].fillna(int(0), inplace=True)
df['rain'].fillna(int(0), inplace=True)
df['snow'].fillna(int(0), inplace=True)

# df['user_check'].fillna(int(0), inplace=True)


# df['sweat'].fillna(int(df['sweat'].mean()), inplace=True)


# df['fragrance_discontinued'].fillna(int(0), inplace=True)
# df['brand_discontinued'].fillna(int(0), inplace=True)


C:\Anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [137]:

# datetime object containing current date and time
# now = datetime.now()
 
# print("now =", now)

# dd/mm/YY H:M:S
# dt_string = now.strftime('%Y-%m-%d 12:00:00')
# print("date and time =", dt_string)	

In [138]:
df['number_of_sprays'] = 7
df['apply_time'] = datetime.now().strftime('%Y-%m-%d 12:00:00')

In [139]:
df = df.convert_dtypes()

# Datetime
df['dob']               = df['dob'].astype('datetime64[ns]')
df['apply_time']        = df['apply_time'].astype('datetime64[ns]')
# df['wear_off_time']     = df['wear_off_time'].astype('datetime64[ns]')

# Boolean
# df['fragrance_discontinued']    = df['fragrance_discontinued'].astype('bool')
# df['brand_discontinued']        = df['brand_discontinued'].astype('bool')

# Categorical Variables
# df["ufr_id"]                = df["ufr_id"].astype(CategoricalDtype(df.ufr_id.unique()))

df["weather_main"]          = df["weather_main"].astype(CategoricalDtype(df.weather_main.unique()))
# df["weather_description"]   = df["weather_description"].astype(CategoricalDtype(df.weather_description.unique()))
df["fp_id"]                 = df["fp_id"].astype(CategoricalDtype(df.fp_id.unique()))
df["gender"]                = df["gender"].astype(CategoricalDtype(df.gender.unique()))
df["profession"]            = df["profession"].astype(CategoricalDtype(df.profession.unique()))
df["skin_type"]             = df["skin_type"].astype(CategoricalDtype(df.skin_type.unique()))
# df["climate"]               = df["climate"].astype(CategoricalDtype(df.climate.unique()))
df["season"]                = df["season"].astype(CategoricalDtype(df.season.unique()))
df["fp_country"]            = df["fp_country"].astype(CategoricalDtype(df.fp_country.unique()))
# df["fp_time_zone"]          = df["fp_time_zone"].astype(CategoricalDtype(df.fp_time_zone.unique()))
# df["fragrance_id"]          = df["fragrance_id"].astype(CategoricalDtype(df.fragrance_id.unique()))
df["fragrance"]             = df["fragrance"].astype(CategoricalDtype(df.fragrance.unique()))
df["fragrance_gender"]      = df["fragrance_gender"].astype(CategoricalDtype(df.fragrance_gender.unique()))
df["fragrance_type"]        = df["fragrance_type"].astype(CategoricalDtype(df.fragrance_type.unique()))
# df["accord"]                = df["accord"].astype(CategoricalDtype(df.accord.unique()))
# df["ingredient"]            = df["ingredient"].astype(CategoricalDtype(df.ingredient.unique()))
# df["brand_id"]              = df["brand_id"].astype(CategoricalDtype(df.brand_id.unique()))
df["brand"]                 = df["brand"].astype(CategoricalDtype(df.brand.unique()))
df["brand_tier"]            = df["brand_tier"].astype(CategoricalDtype(df.brand_tier.unique()))
# df["bo_location_country"]   = df["bo_location_country"].astype(CategoricalDtype(df.bo_location_country.unique()))
# df["bo_location_zone"]      = df["bo_location_zone"].astype(CategoricalDtype(df.bo_location_zone.unique()))


df.dtypes

fragrance                 category
fragrance_gender          category
fragrance_type            category
brand                     category
brand_tier                category
fp_id                     category
gender                    category
dob                 datetime64[ns]
sweat                        Int64
height                       Int64
weight                       Int64
profession                category
skin_type                 category
season                    category
fp_country                category
temp                       Float64
hum                          Int64
dew_point                  Float64
uv_index                   Float64
temp_feels_like            Float64
atm_pressure               Float64
clouds                     Float64
visibility                   Int64
wind_speed                 Float64
rain                       Float64
snow                         Int64
weather_main              category
number_of_sprays             Int64
apply_time          

In [140]:
# Calcualting Age

now = pd.to_datetime('now')
df['age'] = (now - df['dob']).dt.total_seconds() / (60*60*24*365.25)
df.drop(['dob'],axis=1, inplace=True)

In [141]:
# Sorting out Dates

# Apply Time
df['apply_time_year']          = df['apply_time'].dt.year
df['apply_time_month']         = df['apply_time'].dt.month
df['apply_time_day']           = df['apply_time'].dt.day
df['apply_time_hour']          = df['apply_time'].dt.hour
df['apply_time_minute']        = df['apply_time'].dt.minute
df['apply_time_weekday_name']  = df['apply_time'].dt.day_name()

# Wear Off Time
# df['wear_off_time_year']          = df['wear_off_time'].dt.year
# df['wear_off_time_month']         = df['wear_off_time'].dt.month
# df['wear_off_time_day']           = df['wear_off_time'].dt.day
# df['wear_off_time_hour']          = df['wear_off_time'].dt.hour
# df['wear_off_time_minute']        = df['wear_off_time'].dt.minute
# df['wear_off_time_weekday_name']  = df['wear_off_time'].dt.day_name()

# Type Cast
df['age']               = df['age'].astype('float')

# Drop Apply Time & Wear Off Time
df.drop(['apply_time'],axis=1, inplace=True)
# df.drop(['wear_off_time'],axis=1, inplace=True)

In [142]:
df.describe()

,sweat,height,weight,temp,hum,dew_point,uv_index,temp_feels_like,atm_pressure,clouds,...,wind_speed,rain,snow,number_of_sprays,age,apply_time_year,apply_time_month,apply_time_day,apply_time_hour,apply_time_minute
count,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1.000000,1.0,1.0,1.0,1.0,1.0
mean,82.0,54.0,54.0,82.975,75.0,74.45125,13.2475,89.745,1010.75,85.75,...,11.01875,2.69375,0.0,7.0,1.446692,2021.0,4.0,23.0,12.0,0.0
std,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
min,82,54,54,82.975,75,74.45125,13.2475,89.745,1010.75,85.75,...,11.01875,2.69375,0,7,1.446692,2021.0,4.0,23.0,12.0,0.0
25%,82.0,54.0,54.0,82.975,75.0,74.45125,13.2475,89.745,1010.75,85.75,...,11.01875,2.69375,0.0,7.0,1.446692,2021.0,4.0,23.0,12.0,0.0
50%,82.0,54.0,54.0,82.975,75.0,74.45125,13.2475,89.745,1010.75,85.75,...,11.01875,2.69375,0.0,7.0,1.446692,2021.0,4.0,23.0,12.0,0.0
75%,82.0,54.0,54.0,82.975,75.0,74.45125,13.2475,89.745,1010.75,85.75,...,11.01875,2.69375,0.0,7.0,1.446692,2021.0,4.0,23.0,12.0,0.0
max,82,54,54,82.975,75,74.45125,13.2475,89.745,1010.75,85.75,...,11.01875,2.69375,0,7,1.446692,2021.0,4.0,23.0,12.0,0.0


In [143]:
# use pd.concat to join the new columns with your original dataframe
# df = pd.concat([df,pd.get_dummies(df['ufr_id'], prefix='ufr_id')],axis=1)

df = pd.concat([df,pd.get_dummies(df['apply_time_weekday_name'], prefix='apply_time_weekday_name')],axis=1)
# df = pd.concat([df,pd.get_dummies(df['wear_off_time_weekday_name'], prefix='wear_off_time_weekday_name')],axis=1)

df = pd.concat([df,pd.get_dummies(df['weather_main'], prefix='weather_main')],axis=1)
# df = pd.concat([df,pd.get_dummies(df['weather_description'], prefix='weather_description')],axis=1)

df = pd.concat([df,pd.get_dummies(df['fp_id'], prefix='fp_id')],axis=1)
df = pd.concat([df,pd.get_dummies(df['gender'], prefix='gender')],axis=1)
df = pd.concat([df,pd.get_dummies(df['profession'], prefix='profession')],axis=1)
df = pd.concat([df,pd.get_dummies(df['skin_type'], prefix='skin_type')],axis=1)
# df = pd.concat([df,pd.get_dummies(df['climate'], prefix='climate')],axis=1)
df = pd.concat([df,pd.get_dummies(df['season'], prefix='season')],axis=1)

df = pd.concat([df,pd.get_dummies(df['fp_country'], prefix='fp_country')],axis=1)
# df = pd.concat([df,pd.get_dummies(df['fp_time_zone'], prefix='fp_time_zone')],axis=1)

# df = pd.concat([df,pd.get_dummies(df['fragrance_id'], prefix='fragrance_id')],axis=1)
df = pd.concat([df,pd.get_dummies(df['fragrance'], prefix='fragrance')],axis=1)
df = pd.concat([df,pd.get_dummies(df['fragrance_gender'], prefix='fragrance_gender')],axis=1)
df = pd.concat([df,pd.get_dummies(df['fragrance_type'], prefix='fragrance_type')],axis=1)

# df = pd.concat([df,pd.get_dummies(df['accord'], prefix='accord')],axis=1)
# df = pd.concat([df,pd.get_dummies(df['ingredient'], prefix='ingredient')],axis=1)

# df = pd.concat([df,pd.get_dummies(df['brand_id'], prefix='brand_id')],axis=1)
df = pd.concat([df,pd.get_dummies(df['brand'], prefix='brand')],axis=1)
df = pd.concat([df,pd.get_dummies(df['brand_tier'], prefix='brand_tier')],axis=1)
# df = pd.concat([df,pd.get_dummies(df['bo_location_country'], prefix='bo_location_country')],axis=1)
# df = pd.concat([df,pd.get_dummies(df['bo_location_zone'], prefix='bo_location_zone')],axis=1)

In [144]:
# now drop the original 'country' column (you don't need it anymore)
# df.drop(['ufr_id'],axis=1, inplace=True)

df.drop(['apply_time_weekday_name'],axis=1, inplace=True)
# df.drop(['wear_off_time_weekday_name'],axis=1, inplace=True)

df.drop(['weather_main'],axis=1, inplace=True)
# df.drop(['weather_description'],axis=1, inplace=True)

df.drop(['fp_id'],axis=1, inplace=True)
df.drop(['gender'],axis=1, inplace=True)
df.drop(['profession'],axis=1, inplace=True)
df.drop(['skin_type'],axis=1, inplace=True)
# df.drop(['climate'],axis=1, inplace=True)
df.drop(['season'],axis=1, inplace=True)

df.drop(['fp_country'],axis=1, inplace=True)
# df.drop(['fp_time_zone'],axis=1, inplace=True)

# df.drop(['fragrance_id'],axis=1, inplace=True)
df.drop(['fragrance'],axis=1, inplace=True)
df.drop(['fragrance_gender'],axis=1, inplace=True)
df.drop(['fragrance_type'],axis=1, inplace=True)

# df.drop(['accord'],axis=1, inplace=True)
# df.drop(['ingredient'],axis=1, inplace=True)

# df.drop(['brand_id'],axis=1, inplace=True)
df.drop(['brand'],axis=1, inplace=True)
df.drop(['brand_tier'],axis=1, inplace=True)
# df.drop(['bo_location_country'],axis=1, inplace=True)
# df.drop(['bo_location_zone'],axis=1, inplace=True)

# To Be Included Later
# df.drop(['indoor_time_percentage'],axis=1, inplace=True)
# df.drop(['number_of_sprays'],axis=1, inplace=True)
# df.drop(['projection'],axis=1, inplace=True)

In [145]:
print(df.shape)
print(df.duplicated(keep='first').sum())

(1, 34)
0


In [146]:
df.head()

,sweat,height,weight,temp,hum,dew_point,uv_index,temp_feels_like,atm_pressure,clouds,...,gender_Male,profession_Cafe Worker,skin_type_Very Oily,season_Spring,fp_country_United States of America,fragrance_Heint,fragrance_gender_Female,fragrance_type_Eau Fraiche,brand_Venom,brand_tier_High
0,82,54,54,82.975,75,74.45125,13.2475,89.745,1010.75,85.75,...,1,1,1,1,1,1,1,1,1,1


In [147]:
# Use this and fix this
# aggregation_functions = {'longevity': 'first', 'amount': 'sum', 'name': 'first'}
# df_new = df.groupby(df['ufr_id']).aggregate(aggregation_functions)
# df_new.head()

In [148]:
# df.drop_duplicates(subset=None, keep="first", inplace=True)
# df.shape

In [149]:
# Columns with any null values
df.columns[df.isna().any()].tolist()

[]

In [150]:
df.describe()

,sweat,height,weight,temp,hum,dew_point,uv_index,temp_feels_like,atm_pressure,clouds,...,gender_Male,profession_Cafe Worker,skin_type_Very Oily,season_Spring,fp_country_United States of America,fragrance_Heint,fragrance_gender_Female,fragrance_type_Eau Fraiche,brand_Venom,brand_tier_High
count,1,1,1,1,1,1,1,1,1,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,82.0,54.0,54.0,82.975,75.0,74.45125,13.2475,89.745,1010.75,85.75,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,82,54,54,82.975,75,74.45125,13.2475,89.745,1010.75,85.75,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,82.0,54.0,54.0,82.975,75.0,74.45125,13.2475,89.745,1010.75,85.75,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,82.0,54.0,54.0,82.975,75.0,74.45125,13.2475,89.745,1010.75,85.75,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,82.0,54.0,54.0,82.975,75.0,74.45125,13.2475,89.745,1010.75,85.75,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,82,54,54,82.975,75,74.45125,13.2475,89.745,1010.75,85.75,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [151]:
df.dtypes

sweat                                    Int64
height                                   Int64
weight                                   Int64
temp                                   Float64
hum                                      Int64
dew_point                              Float64
uv_index                               Float64
temp_feels_like                        Float64
atm_pressure                           Float64
clouds                                 Float64
visibility                               Int64
wind_speed                             Float64
rain                                   Float64
snow                                     Int64
number_of_sprays                         Int64
age                                    float64
apply_time_year                          int64
apply_time_month                         int64
apply_time_day                           int64
apply_time_hour                          int64
apply_time_minute                        int64
apply_time_we

# Model

## Train

In [152]:
with open('longevity_model.pickle', 'rb') as f:
    longevity_model = pickle.load(f)

In [153]:
longevity_model

DecisionTreeRegressor(random_state=0)

In [154]:
y_pred = longevity_model.predict(df)

ValueError: X has 34 features, but DecisionTreeRegressor is expecting 56 features as input.

In [ ]:
df.dtypes

In [ ]:
# define the target variable (dependent variable) as y
y = df.longevity
y=y.astype('int')

# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion='entropy')

In [ ]:
# Train Decision Tree Classifer
model = clf.fit(X_train, y_train)

## Test

In [ ]:
#Predict the response for test dataset
y_pred = model.predict(X_test)

In [ ]:
# Test
# y_pred = lm.predict(X_test)
# from sklearn.model_selection import cross_val_predict
# predicted = cross_val_predict(lm, df_long, y, cv=10)

# fig, ax = plt.subplots()
# ax.scatter(y, predicted)
# ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
# ax.set_xlabel('Measured')
# ax.set_ylabel('Predicted')
# plt.show()

## Evaluate

In [ ]:
## The line / model
plt.scatter(y_test, y_pred)
plt.xlabel("True Values")
plt.ylabel("Predictions")

In [ ]:
# The coefficients
# print('Coefficients: \n', model.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))

# Plot outputs
# plt.scatter(X_test, y_test,  color='black')
# plt.plot(X_test, y_pred, color='blue', linewidth=3)

# plt.xticks(())
# plt.yticks(())

# plt.show()

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

## Save the model in pickle

In [ ]:
#Pickling the list
file = open('longevity_model.pickle', 'wb')

# dump information to that file
list_pickle = pickle.dump(model, file)

# close the file
file.close()


In [ ]:
# Open file to 
file = open('longevity_model.pickle', 'rb')

# Load model from the file
loaded_pickle = pickle.load(file)

# close the file
file.close()

In [ ]:
loaded_pickle

# Write Column Names

In [ ]:
def listToString(s): 
    
    # initialize an empty string
    str1 = "\n" 
    
    # return string  
    return (str1.join(s))

In [ ]:
file = open('column_names.txt', 'w')

# dump information to that file
file.write(listToString(df.columns))

# close the file
file.close()

# END